In [1]:
import pandas as pd

### Download the data and load it to Pandas. 

You can find the data files in the README.

In [6]:
# titles = pd.read_csv('data/titles.csv')
titles = pd.read_csv('titles.csv')
# titles.head()
titles.head()

,title,year
0,The Rising Son,1990
1,The Thousand Plane Raid,1969
2,Crucea de piatra,1993
3,Country,2000
4,Gaiking II,2011


In [2]:
# cast = pd.read_csv('data/cast.csv', index_col=None)
cast = pd.read_csv('cast.csv', index_col=None)
# cast.head()
cast.head()

,title,year,name,type,character,n
0,Closet Monster,2015,Buffy #1,actor,Buffy 4,NaN
1,Suuri illusioni,1985,Homo $,actor,Guests,22.0
2,Battle of the Sexes,2017,$hutter,actor,Bobby Riggs Fan,10.0
3,Secret in Their Eyes,2015,$hutter,actor,2002 Dodger Fan,NaN
4,Steve Jobs,2015,$hutter,actor,1988 Opera House Patron,NaN


In [3]:
# release_dates = pd.read_csv('data/release_dates.csv', index_col=None, parse_dates=['date'], infer_datetime_format=True)
release_dates = pd.read_csv('release_dates.csv', index_col=None, parse_dates=['date'], infer_datetime_format=True)
# release_dates.head()
release_dates.head()

C:\Users\a7902\AppData\Local\Temp\ipykernel_22212\1742529517.py:2: FutureWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  release_dates = pd.read_csv('release_dates.csv', index_col=None, parse_dates=['date'], infer_datetime_format=True)


,title,year,country,date
0,"#73, Shaanthi Nivaasa",2007,India,2007-06-15
1,#Beings,2015,Romania,2015-01-29
2,#Declimax,2018,Netherlands,2018-01-21
3,#Ewankosau saranghaeyo,2015,Philippines,2015-01-21
4,#Horror,2015,USA,2015-11-20


### 1. Define a year as a "Superman year" whose films feature more Superman characters than Batman. How many years in film history have been Superman years? 

#### Use .apply() to make the comparison between number of movies in each year

> Hint:

> Make two dataframes, one for superman roles and one for batman roles. Find the number of roles for each in each year, then merge them so that you have year, n_batman, n_superman. Then use an apply function to check the condition.

In [10]:
# Filter cast DataFrame for Superman and Batman roles
superman_roles = cast[cast['character'] == 'Superman']
batman_roles = cast[cast['character'] == 'Batman']

# Group by year and count number of roles for each character
superman_counts = superman_roles.groupby(superman_roles['year']).size().reset_index(name='n_superman')
batman_counts = batman_roles.groupby(batman_roles['year']).size().reset_index(name='n_batman')

# Merge the counts DataFrames on 'year'
roles_count_per_year = pd.merge(superman_counts, batman_counts, on='year', how='outer').fillna(0)

# Function to determine if a year is a Superman year
def is_superman_year(row):
    return row['n_superman'] > row['n_batman']

# Apply the function to check for Superman years
roles_count_per_year['is_superman_year'] = roles_count_per_year.apply(is_superman_year, axis=1)

# Count the number of True values (Superman years)
num_superman_years = roles_count_per_year['is_superman_year'].sum()

print(f"Number of Superman years in film history: {num_superman_years}")

Number of Superman years in film history: 11


### 2. Classify movies based on the number of roles into 'Small Cast' (1-5 roles), 'Medium Cast' (6-15 roles), and 'Large Cast' (more than 15 roles)

> Hint: Group by title and year, then use a function and .apply() to classify. 

In [11]:
# Group by title and year, and count the number of roles
roles_per_movie = cast.groupby(['title', 'year']).size().reset_index(name='n_roles')

# Function to classify movies based on number of roles
def classify_movie(n_roles):
    if n_roles <= 5:
        return 'Small Cast'
    elif n_roles <= 15:
        return 'Medium Cast'
    else:
        return 'Large Cast'

# Apply the classification function to each movie
roles_per_movie['cast_size'] = roles_per_movie['n_roles'].apply(classify_movie)

# Display the classified movies
print(roles_per_movie.head())

                   title  year  n_roles    cast_size
0       #1 Serial Killer  2013       30   Large Cast
1                     #5  2013        2   Small Cast
2             #50Fathers  2015       40   Large Cast
3                    #66  2015       14  Medium Cast
4  #73, Shaanthi Nivaasa  2007       14  Medium Cast


### 3. Count the number of movies where the same actor or actress played multiple roles. Use .apply() to check if the same name is repeated in any movie's cast.

In [12]:
# Function to check if an actor played multiple roles in a movie
def has_multiple_roles(cast_list):
    cast_counts = cast_list.value_counts()  # Count occurrences of each actor/actress
    return cast_counts[cast_counts > 1].any()  # Check if any actor appeared more than once

# Group by movie title and apply the function to check for multiple roles
movies_with_multiple_roles = cast.groupby('title')['name'].apply(has_multiple_roles)

# Count the number of movies where an actor played multiple roles
num_movies_multiple_roles = movies_with_multiple_roles.sum()

print(f"Number of movies where the same actor played multiple roles: {num_movies_multiple_roles}")

Number of movies where the same actor played multiple roles: 24487


### 4. Find movies with at least one character with a name starting with "Dr.". 

### Use apply to create a new column indicating whether a movie has such a character, and then count the number of these movies.

In [14]:
# Function to check if a movie has a character with a name starting with "Dr."
def has_dr_character(cast_list):
    if isinstance(cast_list, list):
        for name in cast_list:
            if isinstance(name, str) and name.startswith("Dr."):
                return True
    return False

# Apply the function to each movie
cast['has_dr_character'] = cast.groupby('title')['name'].transform(has_dr_character)

# Count the number of movies where at least one character has a name starting with "Dr."
num_movies_with_dr_character = cast.loc[cast['has_dr_character'], 'title'].nunique()

print(f"Number of movies with at least one character with a name starting with 'Dr.': {num_movies_with_dr_character}")

ValueError: Cannot mask with non-boolean array containing NA / NaN values

### 5. Create a new column indicating whether each movie has a lead role (n=1) played by an actress. 

#### Use apply to achieve this, and then count the number of such movies.

In [16]:
# Function to check if a movie has a lead role (n=1) played by an actress
def has_lead_actress_role(cast_list):
    # Check if the cast list is a list and has at least one lead role played by an actress
    if isinstance(cast_list, list):
        lead_actress_count = sum(1 for actor in cast_list if actor['n'] == 1 and actor['type'] == 'actress')
        return lead_actress_count > 0
    return False  # Return False for NaN values or non-list entries

# Apply the function to each movie
cast['has_lead_actress_role'] = cast.groupby('title').apply(lambda x: has_lead_actress_role(x[['n', 'type']].to_dict('records')))

# Count the number of movies where at least one lead role is played by an actress
num_movies_with_lead_actress_role = cast.loc[cast['has_lead_actress_role'], 'title'].nunique()

print(f"Number of movies with at least one lead role played by an actress: {num_movies_with_lead_actress_role}")

ValueError: Cannot mask with non-boolean array containing NA / NaN values